In [ ]:
import os
import numpy as np
import pandas as pd
train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
train.head()

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mb = MultiLabelBinarizer().fit(train.labels.apply(lambda x : x.split()))
labels = pd.DataFrame(mb.transform(train.labels.apply(lambda x : x.split())), columns = mb.classes_)
new_train = pd.concat([train['image'], labels], axis=1)
new_train.head()

In [ ]:
seed = 72 
target_size = (384, 384)
batch_size = 16
resize_img = '../input/resized-plant2021/img_sz_384'
test_img = '../input/plant-pathology-2021-fgvc8/test_images'
submission = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
submission.head()

In [ ]:
'''
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
            rescale = 1/255.,
            rotation_range = 15,
            width_shift_range=0.2, 
            height_shift_range=0.2,
            zoom_range = 0.2,
            shear_range=0.3,
            featurewise_center=True,
            featurewise_std_normalization=True,
            horizontal_flip=True,
            vertical_flip=True,
            validation_split= 0.2,)
test_datagen = ImageDataGenerator(rescale = 1./255)
'''

In [ ]:
'''
train_generator = train_datagen.flow_from_dataframe(
                  new_train,
                  directory = resize_img, 
                  x_col = 'image',
                  y_col = new_train.columns.tolist()[1:],
                  subset = "training",
                  color_mode = "rgb",
                  target_size = target_size,
                  class_mode = "raw",
                  batch_size = batch_size,
                  shuffle = True,
                  seed = seed,)
valid_generator = train_datagen.flow_from_dataframe(
                  new_train,
                  directory = resize_img,
                  x_col = 'image',
                  y_col = new_train.columns.tolist()[1:],
                  subset = "validation",
                  color_mode = "rgb",
                  target_size = target_size,
                  class_mode = "raw",
                  batch_size = batch_size,
                  shuffle = True,
                  seed = seed,)
test_generator = test_datagen.flow_from_dataframe(
                  submission,
                  directory = test_img,
                  x_col = 'image',
                  y_col = 'labels',
                  class_mode = None,
                  batch_size=1,
                  target_size = target_size,
                  color_mode="rgb",
                  shuffle = False,
                  seed = seed,)
''' 

In [ ]:
'''
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import Xception
from tensorflow.keras import layers, optimizers
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from keras import Model
base_model = Xception(include_top=False, 
                weights='imagenet',
                pooling='avg', 
                input_shape=(384,384,3))
base_model.trainable = False
'''

In [ ]:
'''
model_pre = tf.keras.Sequential([
        base_model,
        layers.BatchNormalization(),
        layers.Dense(64,activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(6,activation='sigmoid')
])
'''

Tatol Train

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
            rescale = 1/255.,
            rotation_range = 15,
            width_shift_range=0.2, 
            height_shift_range=0.2,
            zoom_range = 0.2,
            shear_range=0.3,
            featurewise_center=True,
            featurewise_std_normalization=True,
            horizontal_flip=True,
            vertical_flip=True,
            validation_split= 0.05,)


In [ ]:

seed = 72 
target_size = (384, 384)
batch_size = 8
train_generator = train_datagen.flow_from_dataframe(
                  new_train,
                  directory = resize_img, 
                  x_col = 'image',
                  y_col = new_train.columns.tolist()[1:],
                  subset = "training",
                  color_mode = "rgb",
                  target_size = target_size,
                  class_mode = "raw",
                  batch_size = batch_size,
                  shuffle = True,
                  seed = seed,
                  )

valid_generator = train_datagen.flow_from_dataframe(
                  new_train,
                  directory = resize_img,
                  x_col = 'image',
                  y_col = new_train.columns.tolist()[1:],
                  subset = "validation",
                  color_mode = "rgb",
                  target_size = target_size,
                  class_mode = "raw",
                  batch_size = batch_size,
                  shuffle = True,
                  seed = seed,)


In [ ]:
import keras
def load_model():
    model_pre = keras.models.load_model("../input/xmodels/Xception_test2.h5")
    return model_pre
model = load_model()

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow as tf
from tensorflow import keras
from keras import Model
model.compile(optimizer=tf.keras.optimizers.Adam(
          learning_rate=0.001),
          loss='binary_crossentropy',
          metrics=[tf.keras.metrics.BinaryAccuracy(name='binary_accuracy'),tf.keras.metrics.AUC(name='auc')])

model_checkpoint = ModelCheckpoint(
            filepath="./Xception_test2.h5", 
            mode='max',
            monitor='val_auc', 
            save_best_only=True, 
            verbose=1)

early_stopping = EarlyStopping(
            monitor='val_auc', 
            min_delta=0,
            patience=5, 
            verbose=1, 
            restore_best_weights=True)

In [ ]:
model = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    callbacks=[model_checkpoint, early_stopping])

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_dataframe(
                  submission,
                  directory = test_img,
                  x_col = 'image',
                  y_col = 'labels',
                  class_mode = None,
                  batch_size=1,
                  target_size = target_size,
                  color_mode="rgb",
                  shuffle = False,
                  seed = seed,)

In [ ]:
model = keras.models.load_model("./Xception_test2.h5")
pred = model.predict(test_generator)

In [ ]:
perdict = (pred>0.44)
n_label = new_train.columns.tolist()[1:]
answer = []

for i in range(perdict.shape[0]):
    temp = []
    for j, k in enumerate(n_label):
        if perdict[i, j]:
            temp.append(k)
    answer.append(temp)
    
answer = [' '.join(n) for n in answer]

In [ ]:
submission['labels'] = np.array(answer)
submission

In [ ]:
submission.to_csv('submission.csv', index=False)